<p style="font-size:24px;">Python ETL for Currency Exchange Data</p>


**Description:**  
This notebook demonstrates a Python-based ETL pipeline to retrieve, process, and merge currency exchange rate data. It includes functions to:

- Fetch the latest exchange rates from an API.  
- Retrieve historic exchange rate data based on a configurable number of days in the past.  
- Merge the latest and historic datasets while handling duplicates.  
- Convert currency rates to GBP using vectorized operations for improved performance and optimized error handling.

The file serves as a practical example for data engineering tasks, showcasing API integration, data transformation with pandas, and effective error handling strategies.

In [1]:

# We will be using API from https://exchangeratesapi.io/ 
# Kindly use this link and sign up to get your free API key 

In [ ]:
# What are APIs in data engineering ? 


In [ ]:
# we will import some libraries 
import pandas as pd # TODO: What is library doing ? 
import requests # TODO: what is library doing ? 
import json # TODO: What is library doing ? 
from datetime import datetime, timedelta # TODO: What is library doing ? 

In [ ]:
# TODO: Make a decision on which one to choose either this or below cell(SIMPLIFIED VERSION)


# 1. FETCHING LATEST EXCHANGE RATES 
# API REQUEST - MAKES AN HTTP GET REQUEST TO THE LATEST END POINT OF AN EXCHANGE RATE API USING ACCESS KEY 
def latest_extraction():
    # "latest" endpoint - request the most recent exchange rate data
    # making an api request and returns data in json 
    latest_response = requests.get("http://api.exchangeratesapi.io/v1/latest?access_key=6e45c8ce217f5beca6893ac6968c5c2f")  # noqa: E501
    # converts it to a text 
    latest_data = latest_response.text
    # JSON PARSING AND DATAFRAME CREATION 
    # TEXT IS PARSED INTO A DICTIONARY 
    # RATES KEY HOLDS CURRENCY RATES WHICH ARE CONVERTED INTO A LIST 
    # IT IS THEN CONVERTED INTO A DATAFRAME WITH COLUMNS CURRENCY AND RATE 
    # DATE COLUMN IS ALSO ADDED BASED ON THE API RESPONSE 
    latest_parsed = json.loads(latest_data)
    latest_data_dict = latest_response.json()['rates']
    latest_data_items = latest_data_dict.items()
    latest_data_list = list(latest_data_items)
    latest_df = pd.DataFrame(latest_data_list, columns=['currency', 'rate'])
    latest_df["date"] = latest_parsed["date"]
    return latest_df


# Step 1: Extract latest exchange rates and display the data frame
print("Step 1: Extracting Latest Exchange Rates")
latest_df = latest_extraction()
display(latest_df.head(10))




Step 1: Extracting Latest Exchange Rates


,currency,rate,date
0,AED,3.843685,2025-02-22
1,AFN,77.423228,2025-02-22
2,ALL,98.991341,2025-02-22
3,AMD,412.463920,2025-02-22
4,ANG,1.886635,2025-02-22
5,AOA,958.556336,2025-02-22
6,ARS,1108.465738,2025-02-22
7,AUD,1.646152,2025-02-22
8,AWG,1.883625,2025-02-22
9,AZN,1.783134,2025-02-22


In [30]:
# SIMPLIFIED VERSION 

def latest_extraction():
    """
    Fetches the latest exchange rate data from the API and returns it as a pandas DataFrame.
    """
    url = "http://api.exchangeratesapi.io/v1/latest?access_key=6e45c8ce217f5beca6893ac6968c5c2f"
    response = requests.get(url)
    response.raise_for_status()  # Raise an error if the request failed
    data = response.json()
    df = pd.DataFrame(data["rates"].items(), columns=["currency", "rate"])
    df["date"] = data["date"]
    return df

# Step 1: Extract latest exchange rates and display the DataFrame
print("Step 1: Extracting Latest Exchange Rates")
latest_df = latest_extraction()
display(latest_df)


Step 1: Extracting Latest Exchange Rates


,currency,rate,date
0,AED,3.843685,2025-02-22
1,AFN,77.423228,2025-02-22
2,ALL,98.991341,2025-02-22
3,AMD,412.463920,2025-02-22
4,ANG,1.886635,2025-02-22
...,...,...,...
165,YER,259.078874,2025-02-22
166,ZAR,19.220967,2025-02-22
167,ZMK,9419.385666,2025-02-22
168,ZMW,29.479376,2025-02-22


In [ ]:
# TODO: make a decision on which one to go with either this or below cell(SIMPLIFIED VERSION)


# 2. FETCHING HISTORIC RATES 
# CALCULATES THE DATE FOR TWO DAYS AGO USING DATETIME.NOW() & TIMEDELTA 
# TODO: HOW IS THAT BEING CALCULATED ?
# API REQUEST AND DATA PROCESSING 
# FETCHES THE DATA IN THE SAME MANNER AS THE LATEST RATES IN ABOVE CODE 
def historic_extraction():
    # "Historic - 2DAYS AGO" endpoint - request the exchange rate data from 2 days ago  # noqa: E501

    # Calculate date for 10 days ago
    N = 10
    date_N_days_ago = datetime.now() - timedelta(days=N)
    date_N_days_ago = date_N_days_ago.strftime('%Y-%m-%d')

    historic_response = requests.get("http://api.exchangeratesapi.io/v1/{}?access_key=6e45c8ce217f5beca6893ac6968c5c2f".format(date_N_days_ago))  # noqa: E501
    historic_data = historic_response.text
    historic_parsed = json.loads(historic_data)
    historic_data_dict = historic_response.json()['rates']
    historic_data_items = historic_data_dict.items()
    historic_data_list = list(historic_data_items)
    historic_df = pd.DataFrame(historic_data_list, columns=['currency', 'rate'])  # noqa: E501
    historic_df["date"] = historic_parsed["date"]

    return historic_df


    # Step 2: Extract historic exchange rates (2 days ago) and display the data frame
print("\nStep 2: Extracting Historic Exchange Rates")
historic_df = historic_extraction()
display(historic_df.head(10))


Step 2: Extracting Historic Exchange Rates


,currency,rate,date
0,AED,3.816741,2025-02-12
1,AFN,76.374656,2025-02-12
2,ALL,99.550110,2025-02-12
3,AMD,412.079041,2025-02-12
4,ANG,1.873143,2025-02-12
5,AOA,947.690481,2025-02-12
6,ARS,1097.703849,2025-02-12
7,AUD,1.654404,2025-02-12
8,AWG,1.870442,2025-02-12
9,AZN,1.761431,2025-02-12


In [32]:
# SIMPLIFIED VERSION 

def historic_extraction(n_days=10):
    """
    Fetches exchange rate data from n_days ago from the API and returns it as a pandas DataFrame.
    
    Parameters:
        n_days (int): Number of days ago for which to fetch historic exchange rates.
        
    Returns:
        pd.DataFrame: A DataFrame containing currency rates and the corresponding date.
    """
    # Calculate the date n_days ago
    date_n_days_ago = (datetime.now() - timedelta(days=n_days)).strftime('%Y-%m-%d')
    
    # Build the URL using an f-string
    url = f"http://api.exchangeratesapi.io/v1/{date_n_days_ago}?access_key=6e45c8ce217f5beca6893ac6968c5c2f"
    
    # Make the API request
    response = requests.get(url)
    response.raise_for_status()  # Raises an HTTPError if the request returned an unsuccessful status code
    
    # Parse the response JSON
    data = response.json()
    
    # Create a DataFrame directly from the rates dictionary items
    df = pd.DataFrame(data["rates"].items(), columns=["currency", "rate"])
    df["date"] = data["date"]
    
    return df

# Step 2: Extract historic exchange rates (10 days ago by default) and display the DataFrame
print("\nStep 2: Extracting Historic Exchange Rates")
historic_df = historic_extraction(n_days=10)
display(historic_df.head(10))



Step 2: Extracting Historic Exchange Rates


,currency,rate,date
0,AED,3.816741,2025-02-12
1,AFN,76.374656,2025-02-12
2,ALL,99.550110,2025-02-12
3,AMD,412.079041,2025-02-12
4,ANG,1.873143,2025-02-12
5,AOA,947.690481,2025-02-12
6,ARS,1097.703849,2025-02-12
7,AUD,1.654404,2025-02-12
8,AWG,1.870442,2025-02-12
9,AZN,1.761431,2025-02-12


In [33]:
# 3. MERGING THE DATAFRAME 
# CONCATENATION -> COMBINES THE TWO DATAFRAME(LATEST AND HISTORIC) 
# SORTING AND REMOVE DUPLICATES 
def merge_latest_and_historic(latest_df, historic_df):

    df1 = latest_df
    df2 = historic_df

    frames = [df1, df2]
    result = pd.concat(frames, ignore_index=True)
    exhange_rates = result.sort_values(by='currency', ascending=True, ignore_index=True)  # noqa: E501
    # Drop duplicate records from dataframe if any
    exhange_rates = exhange_rates.drop_duplicates(keep=False)

    return exhange_rates

    # Step 3: Merge the latest and historic data and display the merged data frame
print("\nStep 3: Merging Latest and Historic Data")
merged_df = merge_latest_and_historic(latest_df, historic_df)
display(merged_df.head(10))



Step 3: Merging Latest and Historic Data


,currency,rate,date
0,AED,3.843685,2025-02-22
1,AED,3.816741,2025-02-12
2,AFN,76.374656,2025-02-12
3,AFN,77.423228,2025-02-22
4,ALL,98.991341,2025-02-22
5,ALL,99.550110,2025-02-12
6,AMD,412.463920,2025-02-22
7,AMD,412.079041,2025-02-12
8,ANG,1.886635,2025-02-22
9,ANG,1.873143,2025-02-12


In [34]:
def add_gbp_conversion(df):
    """
    Adds a new column 'rate_in_GBP' to the DataFrame by converting each currency's rate into GBP.
    The conversion is done by dividing each rate by the GBP rate for the corresponding date.
    
    Parameters:
        df (pd.DataFrame): Merged DataFrame with columns 'currency', 'rate', and 'date'.
        
    Returns:
        pd.DataFrame: The input DataFrame with an additional 'rate_in_GBP' column.
    """
    # Create a Series mapping each date to its GBP rate.
    # We drop duplicates in case the GBP rate appears more than once per date.
    gbp_rates = (
        df.loc[df["currency"] == "GBP", ["date", "rate"]]
        .drop_duplicates(subset="date")
        .set_index("date")["rate"]
    )
    
    # Define a function that applies the conversion for each row.
    def convert_row(row):
        gbp_rate = gbp_rates.get(row["date"])
        return row["rate"] / gbp_rate if gbp_rate else None
    
    # Apply the conversion function to each row.
    df["rate_in_GBP"] = df.apply(convert_row, axis=1)
    return df
print("\nAdding 'rate_in_GBP' column to the merged DataFrame")
merged_df = add_gbp_conversion(merged_df)
display(merged_df.head(10))




Adding 'rate_in_GBP' column to the merged DataFrame


,currency,rate,date,rate_in_GBP
0,AED,3.843685,2025-02-22,4.639235
1,AED,3.816741,2025-02-12,4.572787
2,AFN,76.374656,2025-02-12,91.503474
3,AFN,77.423228,2025-02-22,93.447965
4,ALL,98.991341,2025-02-22,119.480157
5,ALL,99.550110,2025-02-12,119.269682
6,AMD,412.463920,2025-02-22,497.833985
7,AMD,412.079041,2025-02-12,493.706499
8,ANG,1.886635,2025-02-22,2.277123
9,ANG,1.873143,2025-02-12,2.244188
